# Cython C++ QASM Simulator

## Using the simulator as a QISKit engine

We now import the cython simulator and use it as a qiskit backend

In [13]:
# Add qiskit development branch to path
# Once qiskit 0.6 is released this can use the pip qiskit
QISKIT_PATH = '~/git/qiskit-terra/'
import sys, os
sys.path.append(os.path.expanduser(QISKIT_PATH))

In [14]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import Cython QASM Simulator Backend
from qiskit_addon_qv import AerQvSimulator

# Backends
aer = AerQvSimulator()
aer.set_max_threads_shot(-1)

In [15]:
def random_circuit(num_qubits, num_gates):
    qr = qiskit.QuantumRegister(num_qubits)
    cr = qiskit.ClassicalRegister(num_qubits)
    circ = qiskit.QuantumCircuit(qr, cr)
    for j in range(num_gates):
        q = np.random.randint(0, num_qubits)
        
        if num_qubits == 1:
            r = np.random.randint(0, 3)
        else:
            r = np.random.randint(0, 3)
        
        if r == 0:
            circ.h(qr[q])
        elif r == 1:
            circ.s(qr[q])
        elif r == 2:
            circ.t(qr[q])
        elif r == 3:       
            circ.cx(qr[q], qr[(q + 1) % num_qubits])
    circ.measure(qr, cr)
    return circ

In [16]:
circ1 = random_circuit(5, 50)
result1 = qiskit.execute(circ1, aer, shots=1000).result()

In [6]:
result1.get_counts(circ)

{'0x0': 6,
 '0x10': 10,
 '0x12': 54,
 '0x18': 71,
 '0x1a': 366,
 '0x2': 62,
 '0x8': 61,
 '0xa': 370}

### Noise simulation

Simulation of an amplitude damping channel on single qubit gates

In [20]:
noise_model = {
    "errors": [
        {
            "type": "kraus",
            "operations": ["h", "t"],
            "probabilities": [1],
            "matrices": [np.array([[1, 0], [0, 0.5]]),
                        np.array([[0, 0.86602540378], [0, 0]])]
        }
    ]
}

In [21]:
aer.load_noise_model(noise_model)

In [22]:
circ2 = random_circuit(5, 50)
result2 = qiskit.execute(circ2, aer, shots=1000).result()
result2.get_data(circ2)

{'counts': {'0x0': 2,
  '0x13': 2,
  '0x17': 9,
  '0x1a': 1,
  '0x1f': 751,
  '0x3': 4,
  '0x4': 1,
  '0x7': 32,
  '0x8': 1,
  '0xd': 1,
  '0xf': 196}}